# Look at Data Available

In [ ]:
import polars as pl
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))
from DataHandler import *
base_dir = os.path.dirname(os.path.abspath(os.getcwd()))
data_dir = os.path.join(base_dir, 'Data')
dtypes_locId = {
    "locId": pl.Utf8  # Read the locId column as a string
}
infer_schema_length = 1000

dh = DataHandler(pl.read_csv(os.path.join(data_dir,"contamezzi_df.csv")),
            pl.read_csv(os.path.join(data_dir,"manifestazioni_df.csv")),
            pl.read_csv(os.path.join(data_dir,"meteotrentino_bollettino_df.csv")),
            pl.read_csv(os.path.join(data_dir,"contamezzi_descrizione_sensore_df.csv")),
            pl.read_csv(os.path.join(data_dir,"contapersone_passaggi_df.csv")),
            pl.read_csv(os.path.join(data_dir,"contapersone_presenze_df.csv")),
            pl.read_csv(os.path.join(data_dir,"statistiche_parcheggi_molveno_df.csv")),
            pl.read_csv(os.path.join(data_dir,"movimento_turistico_df.csv")),
            pl.read_csv(os.path.join(data_dir,"extra_strutture_df.csv")),
            pl.read_csv(os.path.join(data_dir,"survey_df.csv")),
            pl.read_csv(os.path.join(data_dir,"vodafone_aree_df.csv"),dtypes=dtypes_locId,infer_schema_length=infer_schema_length),
            pl.read_csv(os.path.join(data_dir,"vodafone_attendences_df.csv"),dtypes=dtypes_locId,infer_schema_length=infer_schema_length),
            pl.read_csv(os.path.join(data_dir,"vodafone_attendences_STR_df.csv"),dtypes=dtypes_locId,infer_schema_length=infer_schema_length),
            pl.read_csv(os.path.join(data_dir,"movimento_turistico_molveno_df.csv"))
            ) 
dh.contamezzi_df = dh.contamezzi_df.join(dh.contamezzi_descrizione_sensore_df, on='sensore', how='left')
dh.CastDataFramesDate()
pandas_df = dh.contamezzi_df.to_pandas()

# Then create the GeoDataFrame from the Pandas DataFrame
gdf = gpd.GeoDataFrame(
    pandas_df, 
    geometry=gpd.points_from_xy(pandas_df["longitudine"], pandas_df["latitudine"]),
    crs=4326
)
gdf.drop(columns=["longitudine", "latitudine","_right"], inplace=True)
gdf.rename(columns={"direzione_right":"direzione"}, inplace=True)
gdf["aggregated_count"] = gdf.apply(lambda x: x["cl1"] + x["cl2"] + x["cl3"] + x["cl4"] + x["cl5"] + x["cl6"] + x["cl7"],axis=1)


In [ ]:
gdf

In [ ]:
import matplotlib.pyplot as plt
plt.hist(gdf["nonrilevato"])

In [ ]:
import contextily as ctx
import matplotlib.pyplot as plt
import geopandas as gpd
import osmnx as ox
from matplotlib_scalebar.scalebar import ScaleBar




gdf_unified = pd.DataFrame()
attendences_foreigners = dh.vodafone_attendences_df.join(dh.vodafone_aree_df, on='locId', how='left')
names_zones_to_consider = attendences_foreigners.filter(pl.col("locName").is_in(["provincia"])).unique("locDescr")["locDescr"].to_list()
columns_to_retain = ["geometry","public_transport","tourism","amenity"]

for comune in names_zones_to_consider:
    fig, ax = plt.subplots(figsize=(10, 10))
    if "+" in comune:
        pass
    else:
        try:
            gdf = ox.geometries_from_place(comune,tags = {'building':True
                                                        })
            gdf_unified = pd.concat([gdf_unified,gdf],ignore_index=True)
            gdf_unified = gpd.GeoDataFrame(gdf_unified , geometry='geometry') 
        except:
            pass
fig,ax = plt.subplots(figsize=(10, 10))
gdf_unified.to_crs(crs_proj).plot(ax=ax, color='red')
ctx.add_basemap(ax=ax,crs= crs_proj,source=ctx.providers.OpenStreetMap.Mapnik,attribution="© OpenStreetMap contributors")
scalebar = ScaleBar(1, location='lower right')
ax.add_artist(scalebar)


In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import inv

np.random.seed(42)

# Parameters
N = 500                 # Number of nodes
p = 0.01                # ER probability
alpha = 0.02            # Attenuation parameter (should be < 1/lambda_max)

# 1. Create sparse undirected graph
G = nx.erdos_renyi_graph(N, p)
A = nx.adjacency_matrix(G).astype(float).toarray()

# 2. Compute Katz centrality via matrix inversion
I = np.eye(N)
try:
    Katz_inv = np.linalg.solve(I - alpha * A, np.ones(N)) - 1
except np.linalg.LinAlgError:
    print("Matrix inversion failed; try a smaller alpha.")

# 3. Implement the cavity method
def cavity_katz(A, alpha, max_iter=100, tol=1e-6):
    N = A.shape[0]
    neighbors = [np.where(A[i])[0] for i in range(N)]

    # Initialize cavity messages: each (i->j) is (mu, V)
    mu_cav = {}
    V_cav = {}
    for i in range(N):
        for j in neighbors[i]:
            mu_cav[(i, j)] = 0.0
            V_cav[(i, j)] = 1.0

    for _ in range(max_iter):
        delta = 0.0
        mu_cav_new = {}
        V_cav_new = {}

        for i in range(N):
            for j in neighbors[i]:
                neigh = [k for k in neighbors[i] if k != j]
                V_new = 1.0 / (1 - alpha**2 * sum(V_cav.get((k, i), 0) for k in neigh))
                mu_new = V_new * (1 + alpha * sum(mu_cav.get((k, i), 0) for k in neigh))

                mu_cav_new[(i, j)] = mu_new
                V_cav_new[(i, j)] = V_new

                delta += abs(mu_cav[(i, j)] - mu_new)

        mu_cav, V_cav = mu_cav_new, V_cav_new
        if delta < tol:
            break

    # Compute marginal means µ_i
    mu = np.zeros(N)
    for i in range(N):
        V_i = 1.0 / (1 - alpha**2 * sum(V_cav[(j, i)] for j in neighbors[i]))
        mu[i] = V_i * (1 + alpha * sum(mu_cav[(j, i)] for j in neighbors[i]))

    Katz_cavity = mu - 1
    return Katz_cavity

Katz_cav = cavity_katz(A, alpha)

# 4. Plot comparison
plt.figure(figsize=(10, 6))
plt.scatter(Katz_inv, Katz_cav, s=10, alpha=0.6, label='Cavity vs Inversion')
plt.plot([Katz_inv.min(), Katz_inv.max()], [Katz_inv.min(), Katz_inv.max()], 'r--', label='y = x')
plt.xlabel("Katz Centrality (Matrix Inversion)")
plt.ylabel("Katz Centrality (Cavity Method)")
plt.title("Katz Centrality: Cavity Method vs Exact Solution")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
dh.vodafone_attendences_df

# Where do we measure fluxes? Three coils 

In [ ]:
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import numpy as np
gdf = gpd.GeoDataFrame(dh.contamezzi_df, geometry=gpd.points_from_xy(dh.contamezzi_df["longitudine"],dh.contamezzi_df["latitudine"]))
crs = "EPSG:4326"
gdf.set_crs(crs, inplace=True)
crs_proj = "EPSG:3857"


center_x, center_y = np.nanmean(gdf.geometry.x), np.nanmean(gdf.geometry.y)
 
print(center_x, center_y)
size = 10000  # Size of the bounding box in map units (meters for EPSG:3857)

# Calculate the bounding box
bbox = [center_x - size, center_y - size, center_x + size, center_y + size]


fig, ax = plt.subplots(figsize=(10, 10))
# Set the bounding box
#ax.set_xlim(bbox[0], bbox[2])
#ax.set_ylim(bbox[1], bbox[3])

gdf.to_crs(crs_proj).plot(ax=ax)
ctx.add_basemap(ax=ax,crs= crs_proj,source=ctx.providers.OpenStreetMap.Mapnik)

In [ ]:
import osmnx as ox
import matplotlib.pyplot as plt
import contextily as ctx
import pandas as pd
from matplotlib_scalebar.scalebar import ScaleBar
#comuni = dh.vodafone_aree_df.filter(pl.col("locName")=="comune")["locDescr"].unique().to_list() 
comuni = ["Molveno","Fai della Paganella"]
columns_to_retain = ["geometry","public_transport","tourism","amenity"]
geometries = [""]
gdf_unified = gpd.GeoDataFrame()
for comune in comuni:
    fig, ax = plt.subplots(figsize=(10, 10))
    gdf = ox.geometries_from_place(comune,tags = {'building':True
                                                  })
    
    gdf_unified = pd.concat([gdf_unified,gdf],ignore_index=True)
    gdf_unified = gpd.GeoDataFrame(gdf_unified , geometry='geometry') 
    gdf_unified = gdf_unified[columns_to_retain]
    gdf.to_crs(crs_proj).plot(ax=ax, color='red')
    ctx.add_basemap(ax=ax,crs= crs_proj,source=ctx.providers.OpenStreetMap.Mapnik)
    plt.show()
fig,ax = plt.subplots(figsize=(10, 10))
gdf_unified.to_crs(crs_proj).plot(ax=ax, color='red')
ctx.add_basemap(ax=ax,crs= crs_proj,source=ctx.providers.OpenStreetMap.Mapnik,attribution="© OpenStreetMap contributors")
scalebar = ScaleBar(1, location='lower right')
ax.add_artist(scalebar)





In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
fig,ax = plt.subplots(figsize=(10, 10))
public_trans_count = Counter(gdf_unified["public_transport"].dropna())
tourism_count = Counter(gdf_unified["tourism"].dropna())
amenity_count = Counter(gdf_unified["amenity"].dropna())
ax.bar(public_trans_count.keys(), public_trans_count.values(), alpha=0.5, label='public_transport')
ax.set_xlabel('publictransport')
ax.set_xticklabels(public_trans_count.keys(), rotation=90)
plt.show()
fig,ax = plt.subplots(figsize=(10, 10))
ax.bar(tourism_count.keys(), tourism_count.values(), alpha=0.5, label='tourism')
ax.set_xlabel('tourism')
ax.set_xticklabels(tourism_count.keys(), rotation=90)
plt.show()
fig,ax = plt.subplots(figsize=(10, 10))
ax.bar(amenity_count.keys(), amenity_count.values(), alpha=0.5, label='amenity')
ax.set_xlabel('amenity')
ax.set_xticklabels(amenity_count.keys(), rotation=90)
plt.show()




# From Boundary to Road Graph

In [ ]:
boundary = gdf_unified.total_bounds
bbox_polygon = bbox(*boundary) 
complete_graph = ox.graph_from_bbox(bbox_polygon,network_type='drive')
complete_graph.plot(ax=ax, node_color='red', node_size=15, edge_color='black')


In [ ]:
from Geometry import *
boundary = gdf_unified.total_bounds
 
grid_area = GenerateGridFromBbox(boundary,crs,crs_proj,Lx = 2.5,Ly = 2)

gdf.to_crs(crs_proj).plot(ax=ax, color='red')
grid_area.to_crs(crs_proj).plot(ax=ax, color='blue',alpha=0.5)
ctx.add_basemap(ax=ax,crs= crs_proj,source=ctx.providers.OpenStreetMap.Mapnik)
plt.show()
fig,ax = plt.subplots(figsize=(10, 10))
gdf_unified.to_crs(crs_proj).plot(ax=ax, color='red')
ctx.add_basemap(ax=ax,crs= crs_proj,source=ctx.providers.OpenStreetMap.Mapnik,attribution="© OpenStreetMap contributors")
scalebar = ScaleBar(1, location='lower right')
ax.add_artist(scalebar)


In [ ]:
#fig,ax = plt.subplots(figsize=(10, 10))
gdf_unified["amenity"].unique()
#ctx.add_basemap(ax=ax,crs= crs_proj,source=ctx.providers.OpenStreetMap.Mapnik,attribution="© OpenStreetMap contributors")

In [ ]:
dh.movimento_turistico_molveno_df

In [ ]:
dh.movimento_turistico_df

In [ ]:
dh.statistiche_parcheggi_molveno_df.filter(pl.col("TOTALE") == pl.col("TOTALE").max())

In [ ]:
dh.movimento_turistico_df

In [ ]:
dh.contapersone_passaggi_df